In [1]:
import pandas as pd


payments = pd.read_csv("data/home_credit/installments_payments.csv")
cash_position = pd.read_csv("data/home_credit/POS_CASH_balance.csv")

In [2]:
# Inspecting the data
payments.head(10)

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.360,6948.360
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525,1716.525
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000,25425.000
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130,24350.130
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040,2160.585
5,1137312,164489,1.0,12,-1384.0,-1417.0,5970.375,5970.375
6,2234264,184693,4.0,11,-349.0,-352.0,29432.295,29432.295
7,1818599,111420,2.0,4,-968.0,-994.0,17862.165,17862.165
8,2723183,112102,0.0,14,-197.0,-197.0,70.740,70.740
9,1413990,109741,1.0,4,-570.0,-609.0,14308.470,14308.470


In [3]:
cash_position.head(10)

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0
5,2207092,342166,-32,12.0,12.0,Active,0,0
6,1110516,204376,-38,48.0,43.0,Active,0,0
7,1387235,153211,-35,36.0,36.0,Active,0,0
8,1220500,112740,-31,12.0,12.0,Active,0,0
9,2371489,274851,-32,24.0,16.0,Active,0,0


In [14]:
# Observing the behavior of a single previous loan
data_single_loan = payments[payments['SK_ID_PREV'] == 1054186]
data_single_loan.sort_values(by=['NUM_INSTALMENT_NUMBER'], inplace=True)

print(data_single_loan)

         SK_ID_PREV  SK_ID_CURR  NUM_INSTALMENT_VERSION  \
577850      1054186      161674                     1.0   
47438       1054186      161674                     1.0   
425089      1054186      161674                     1.0   
3612113     1054186      161674                     1.0   
1878907     1054186      161674                     1.0   
0           1054186      161674                     1.0   
1128187     1054186      161674                     1.0   
138614      1054186      161674                     1.0   
1608829     1054186      161674                     1.0   
1038640     1054186      161674                     1.0   
1402152     1054186      161674                     1.0   
216905      1054186      161674                     1.0   

         NUM_INSTALMENT_NUMBER  DAYS_INSTALMENT  DAYS_ENTRY_PAYMENT  \
577850                       1          -1330.0             -1338.0   
47438                        2          -1300.0             -1307.0   
425089             

<ipython-input-14-cd6b90688944>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_single_loan.sort_values(by=['NUM_INSTALMENT_NUMBER'], inplace=True)


In [16]:
cash_position_single_loan = cash_position[cash_position['SK_ID_PREV'] == 2714724]

print(cash_position_single_loan)

         SK_ID_PREV  SK_ID_CURR  MONTHS_BALANCE  CNT_INSTALMENT  \
686245      2714724      167756             -43             4.0   
1860851     2714724      167756             -45             8.0   
2753540     2714724      167756             -44             8.0   
7673576     2714724      167756             -47             8.0   
8660962     2714724      167756             -46             8.0   

         CNT_INSTALMENT_FUTURE NAME_CONTRACT_STATUS  SK_DPD  SK_DPD_DEF  
686245                     0.0            Completed       0           0  
1860851                    2.0               Active      15          15  
2753540                    1.0               Active      16          16  
7673576                    4.0               Active       0           0  
8660962                    3.0               Active       0           0  


In [23]:
contracts_in_default = cash_position.query("SK_DPD >= 30")

print(f'Total number of contracts in default area {contracts_in_default.SK_ID_PREV.nunique()}')

Total number of contracts in default area 8678


In [17]:
# Build the sequence of payments days and the next payment installment situation

# Para cada SK_ID_PREV:
### ordenar os pagamentos
### montar a sequencia de pagamentos ate t-1 e o resultado em t
### e.g: (para SK_ID_PREV == 1054186)
#### [8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 7 OR On-time
#### [8, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0] 5 OR On-time
#### [8, 7, 5, 0, 0, 0, 0, 0, 0, 0, 0] 7 OR On-time
#### [8, 7, 5, 7, 0, 0, 0, 0, 0, 0, 0] 7 OR On-time
#### [8, 7, 5, 7, 7, 0, 0, 0, 0, 0, 0] 7 OR On-time
#### [8, 7, 5, 7, 7, 7, 0, 0, 0, 0, 0] 25 OR On-time
#### [8, 7, 5, 7, 7, 7, 25, 0, 0, 0, 0] 55 OR Late
#### [8, 7, 5, 7, 7, 7, 25, 55, 0, 0, 0] 44 OR Late
#### [8, 7, 5, 7, 7, 7, 25, 55, 44, 0, 0] 35 OR Late
#### [8, 7, 5, 7, 7, 7, 25, 55, 44, 35, 0] 33 OR Late
#### [8, 7, 5, 7, 7, 7, 25, 55, 44, 35, 33] 33 OR Late

In [ ]:
# Function to calculate payment status
def payment_status(days_between_payment_due_date):
    if days_between_payment_due_date < -30:
        return "Advance payment"
    if -30 < days_between_payment_due_date <= 0:
        return "Payment on schedule"
    if 0 < days_between_payment_due_date <= 30:
        return "Late payment within 30 days"
    if 30 < days_between_payment_due_date <= 90:
        return "Late payment 30+ days but no default"
    if days_between_payment_due_date > 90:
        return "Default"

In [ ]:

# Write as a generator
def get_payments_history(payments_df, cash_position_df, sk_id_prev):
    
    # Filter sk_id_prev in payments_df
    loan_payments = payments_df[payments_df.SK_ID_PREV == sk_id_prev]
    loan_payments.sort_values(by=['NUM_INSTALMENT_NUMBER'], inplace=True)
    loan_payments["payments_due_days"] = (loan_payments.DAYS_ENTRY_PAYMENT - loan_payments.DAYS_INSTALMENT)
    loan_payments["payment_status"] = loan_payments["payments_due_days"].apply(lambda x: payments_status(x))
    loan_payments = loan_payments[["payments_due_days", "payments_status"]]
    
    
    # Merge with cash_position_df info to get the full overview of the contract
    
    
    
    ### ordenar os pagamentos
    ### montar a sequencia de pagamentos ate t-1 e o resultado em t
    ### e.g: (para SK_ID_PREV == 1054186)
    #### [8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 7 OR On-time
    #### [8, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0] 5 OR On-time
    #### [8, 7, 5, 0, 0, 0, 0, 0, 0, 0, 0] 7 OR On-time
    #### [8, 7, 5, 7, 0, 0, 0, 0, 0, 0, 0] 7 OR On-time
    #### [8, 7, 5, 7, 7, 0, 0, 0, 0, 0, 0] 7 OR On-time
    #### [8, 7, 5, 7, 7, 7, 0, 0, 0, 0, 0] 25 OR On-time
    #### [8, 7, 5, 7, 7, 7, 25, 0, 0, 0, 0] 55 OR Late
    #### [8, 7, 5, 7, 7, 7, 25, 55, 0, 0, 0] 44 OR Late
    #### [8, 7, 5, 7, 7, 7, 25, 55, 44, 0, 0] 35 OR Late
    #### [8, 7, 5, 7, 7, 7, 25, 55, 44, 35, 0] 33 OR Late
    #### [8, 7, 5, 7, 7, 7, 25, 55, 44, 35, 33] 33 OR Late
    